<a href="https://colab.research.google.com/github/Kirilenkov/docx_semantic_similarity/blob/main/Cozyduke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Сервис автоматического анализа текста двух документов и присвоения статуса соответсвия
<hr>


Над проектом работает:
Команда **'COZYDUKE'.**

0. Кирсанов Вадим Вадимович
1. Казаченко Екатерина Александровна
2. Кириленков Кирилл Владимирович
3. Гайсина Лиана Ильдаровна
4. Федеряев Клим Александрович
5. Бадретдинова Рушания Ринатовна



In [10]:
# @title Дополнительные модули
# Установка дополнительных модулей
%pip install python-docx
%pip install POT
%pip install torch

# Импорт библиотек и их модулей
import re
from io import BytesIO

import docx
import gensim.downloader as api
import numpy as np
import pandas as pd
import requests
import torch
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity



# 1. Подготовка данных
* **Извлечение данных:** Данные были извлечены из файлов в формате Excel (для анализа требований) и DOCX (для анализа технических характеристик). Содержимое DOCX файла, включающее тексты параграфов и заголовки, было загружено с использованием библиотеки python-docx.

* **Предобработка текста**: Произведена предобработка текстовых данных, включая удаление знаков пунктуации, приведение всех букв к нижнему регистру и удаление лишних пробелов. Эти шаги были выполнены с помощью регулярных выражений и методов обработки строк в Python.
* **Создание словарей и датафреймов:** На основе данных из Excel файла был создан словарь требований. Также был составлен датафрейм для хранения информации о параграфах из DOCX файла, что обеспечило удобную структуру для последующего анализа.

**Примечание**: В дальнейшем планируется улучшить предобработку текста. А также доработать код для извлечения информации из каждого пункта документа.

In [11]:
# @title Создаем словарь с требованиями из файла exel.
# @title Для корректного выполнения кода вы можете заменить путь к файлу на свой в переменной file_path

# Загрузка файла Excel
file_url = 'https://docs.google.com/spreadsheets/d/18wM88vqURx4rKCaeQuf5BTbAF9Uvr2T0/export?format=xlsx'
response = requests.get(file_url)
data = pd.read_excel(BytesIO(response.content))

# Выделение первых двух колонок и их предобработка
data_clean = data[['Пункт СТО ИНТИ', 'Формулировка в СТО ИНТИ']]

# Удаление переноса строк, лишних пробелов, приведение к нижнему регистру и удаление знаков пунктуации
data['Формулировка в СТО ИНТИ'] = data['Формулировка в СТО ИНТИ'].apply(
    lambda x: re.sub(r'\s+', ' ', re.sub(r'[^\w\s]', '', str(x).lower()).replace('\n', ' ').strip())
)

# Создание словаря
items_dict = data_clean.set_index('Пункт СТО ИНТИ')['Формулировка в СТО ИНТИ'].to_dict()

# Вывод первых пяти элементов словаря для проверки
list(items_dict.items())[:5]


[('6.1.1',
  'Насосное оборудование (включая вспомогательные системы), на которое распространяется настоящий стандарт, должно конструироваться и изготавливаться в расчете на срок службы не менее 20 лет (за исключением естественно изнашиваемых деталей, согласно таблице 14) и не менее 3 лет непрерывной эксплуатации. Остановка оборудования для выполнения техобслуживания или проверки не является нарушением\n этого требования.'),
 ('6.1.3.7',
  'Поскольку в герметичных насосах для охлаждения и смазки подшипников\n используется перекачиваемая среда или другая жидкость, она должна оставаться стабильной при прохождении через подшипники. Поставщик должен обеспечить\n работоспособность с учетом изменения температуры и\n давления жидкости циркулирующей в насосе и полости ротора. Работоспособность должна быть обеспечена при максимальной заданной рабочей температуре для минимального стабильного расхода,\n нормальных (рабочих) условий и максимального номинального расхода. Поставщик должен также пред

In [12]:
# @title Создаем датафрейм с техническими характеристиками по абзацам

# Загрузка документа Word
docx_url = 'https://docs.google.com/document/d/1vurxVLf_PuCRjkAzXpYmCPWK4EqJonRt/export?format=docx'
response = requests.get(docx_url)
doc = docx.Document(BytesIO(response.content))

# Чтение всех параграфов в документе, добавление их в список с идентификацией номера пункта или заголовка
all_paragraphs = []
current_header = "Без заголовка"  # Начальный заголовок для текста без явного разделения

for para in doc.paragraphs:
    if para.text.strip():  # Проверка на непустой текст
        # Ищем пункты или заголовки, определяем по форматированию или содержимому
        if para.style.name.startswith('Heading') or (para.text.split()[0].replace('.', '').isdigit() and len(para.text.split()) > 1):
            current_header = para.text.split(maxsplit=1)[0] if para.text.split()[0].replace('.', '').isdigit() else para.text
        all_paragraphs.append((current_header, para.text))

# Создание DataFrame со всем текстом и заголовками
df_all_text = pd.DataFrame(all_paragraphs, columns=['Заголовок', 'Текст параграфа'])

# Предобработка текста: приведение к нижнему регистру, удаление знаков пунктуации и лишних пробелов
df_all_text['Текст параграфа'] = df_all_text['Текст параграфа'].apply(
    lambda x: re.sub(r'[^\w\s]', '', x.lower()).replace('\n', ' ').strip())
df_all_text['Текст параграфа'] = df_all_text['Текст параграфа'].apply(
    lambda x: re.sub(r'\s+', ' ', x))

# Вывод первых пяти строк обработанного DataFrame для проверки
df_all_text.head(40)

,Заголовок,Текст параграфа
0,Без заголовка,общие технические требования
1,Без заголовка,содержание
2,Введение,введение
3,Введение,использование настоящего стандарта в конкретны...
4,Введение,настоящий стандарт является модифицированным п...
5,Введение,дополнительные положения и требования а также ...
6,Введение,знак в начале параграфа или его раздела указыв...
7,Введение,из соображений удобства и в информационных цел...
8,Область применения,область применения
9,Область применения,настоящий стандарт устанавливает требования к ...


In [13]:
# Разделение строки "Заголовок" на номер пункта и чистый заголовок, если номер пункта присутствует

# Добавление нового столбца для номера пункта
df_all_text['Номер пункта'] = None

# Разбиение текущего заголовка на возможный номер пункта и остальную часть
for index, row in df_all_text.iterrows():
    header_text = row['Заголовок']
    # Проверяем, начинается ли заголовок с числовой последовательности, которая может быть номером пункта
    if header_text.split()[0].replace('.', '').isdigit():
        # Извлекаем номер пункта и обновляем заголовок
        split_header = header_text.split(maxsplit=1)
        df_all_text.at[index, 'Номер пункта'] = split_header[0]
        df_all_text.at[index, 'Заголовок'] = split_header[1] if len(split_header) > 1 else ""  # Если есть что-то после номера пункта

# Вывод обновленного DataFrame с отдельным столбцом для номера пункта
df_all_text.head()

,Заголовок,Текст параграфа,Номер пункта
0,Без заголовка,общие технические требования,None
1,Без заголовка,содержание,None
2,Введение,введение,None
3,Введение,использование настоящего стандарта в конкретны...,None
4,Введение,настоящий стандарт является модифицированным п...,None


In [14]:
# Фильтрация DataFrame для поиска строк с определенными номерами пунктов
df_with_numbers = df_all_text[df_all_text['Номер пункта'].notna()]

# Вывод строк с номерами пунктов
df_with_numbers.head()


,Заголовок,Текст параграфа,Номер пункта
212,,6131 заказчик должен указать условия эксплуата...,6.1.3.1
213,,заказчик должен предоставить поставщику паспор...,6.1.3.1
214,,параметры перекачиваемой среды имеют решающее ...,6.1.3.1
215,,важно чтобы перекачиваемая жидкость оставалась...,6.1.3.1
216,,правильная конструкция и выбор насоса зависят ...,6.1.3.1


# 2. Векторизация текста и расчёт сходства
* **Векторизация текста**: Используем TF-IDF векторизацию для преобразования текстов требований и параграфов в числовые векторы и подготовки данных к анализу сходства.

* **Расчет косинусного сходства**: Вычисляем косинусное сходство между векторами параграфов и требований для определения степени соответствия между ними.

In [15]:
# Подготовка текста: соберем все тексты параграфов в список
texts = df_all_text['Текст параграфа'].tolist()

# Подготовка списка текстов требований из предыдущей предобработки (словаря требований)
requirement_texts = list(items_dict.values())

# Объединение текстов требований и текстов параграфов для обучения одного TF-IDF векторизатора
combined_texts = texts + requirement_texts

# Создание и обучение TF-IDF векторизатора на объединенном корпусе
combined_tfidf_vectorizer = TfidfVectorizer()
combined_tfidf_matrix = combined_tfidf_vectorizer.fit_transform(combined_texts)

# Разделение матрицы TF-IDF на часть параграфов и часть требований
tfidf_paragraphs = combined_tfidf_matrix[:len(texts), :]
tfidf_requirements = combined_tfidf_matrix[len(texts):, :]

# Расчет косинусного сходства между параграфами и требованиями
cosine_similarities = cosine_similarity(tfidf_paragraphs, tfidf_requirements)

# Вывод формы матрицы сходств для проверки
cosine_similarities.shape


(2069, 16)

#3. Определение наилучших совпадений
* **Анализ сходства:** Для каждого параграфа определяем требование с максимальным уровнем сходства и выявляем наиболее релевантные соответствия.
* **Создание итогового отчета:** Результаты анализа собраем в датафрейме, который включает информацию о параграфах, соответствующих требованиях и степени их сходства.

In [16]:
# Нахождение индекса требования с максимальным сходством для каждого параграфа
max_similarity_indices = np.argmax(cosine_similarities, axis=1)

# Создание списка для отображения наивысшего соответствия для каждого параграфа
best_match_info = []
for i, index in enumerate(max_similarity_indices):
    paragraph_text = texts[i]
    best_requirement_text = requirement_texts[index]
    similarity_score = cosine_similarities[i, index]
    best_match_info.append((paragraph_text[:50] + '...', best_requirement_text[:50] + '...', similarity_score))

# Пример вывода для первых 5 параграфов
best_match_info[:5]

[('общие технические требования...',
  'Насосное оборудование (включая вспомогательные сис...',
  0.05458044746464749),
 ('содержание...',
  'Насосное оборудование (включая вспомогательные сис...',
  0.0),
 ('введение...', 'Насосное оборудование (включая вспомогательные сис...', 0.0),
 ('использование настоящего стандарта в конкретных ус...',
  'Конструкция насоса должна предусматривать возможно...',
  0.03097176941606686),
 ('настоящий стандарт является модифицированным по от...',
  'Насосное оборудование (включая вспомогательные сис...',
  0.09824599611953541)]

In [17]:
# Создание DataFrame с наилучшими совпадениями для каждого требования
best_match_df = pd.DataFrame(columns=['Номер требования', 'Текст требования', 'Заголовок параграфа', 'Пункт параграфа', 'Текст параграфа', 'Сходство'])

# Перебор каждого требования и нахождение параграфа с максимальным сходством
for req_index, req_text in enumerate(requirement_texts):
    # Индекс параграфа с максимальным сходством для текущего требования
    paragraph_index = np.argmax(cosine_similarities[:, req_index])
    # Соответствующий максимальный сходственный параграф
    max_similarity = cosine_similarities[paragraph_index, req_index]
    # Данные параграфа
    paragraph_text = df_all_text.loc[paragraph_index, 'Текст параграфа']
    paragraph_header = df_all_text.loc[paragraph_index, 'Заголовок']
    paragraph_number = df_all_text.loc[paragraph_index, 'Номер пункта']

# Инициализация пустого DataFrame
best_match_df = pd.DataFrame(columns=['Пункт требования', 'Текст требования', 'Заголовок параграфа', 'Пункт параграфа', 'Текст параграфа', 'Сходство'])

# Сбор всех строк для добавления
rows_to_add = []
for req_key, req_text in items_dict.items():
    paragraph_index = np.argmax(cosine_similarities[:, list(items_dict.keys()).index(req_key)])
    max_similarity = cosine_similarities[paragraph_index, list(items_dict.keys()).index(req_key)]
    paragraph_text = df_all_text.loc[paragraph_index, 'Текст параграфа']
    paragraph_header = df_all_text.loc[paragraph_index, 'Заголовок']
    paragraph_number = df_all_text.loc[paragraph_index, 'Номер пункта']

    # Создание временного DataFrame для текущей строки
    temp_df = pd.DataFrame({
        'Пункт требования': [req_key],
        'Текст требования': [req_text],
        'Заголовок параграфа': [paragraph_header],
        'Пункт параграфа': [paragraph_number],
        'Текст параграфа': [paragraph_text],
        'Сходство': [max_similarity]
    })
    rows_to_add.append(temp_df)

# Добавление всех собранных строк в основной DataFrame
best_match_df = pd.concat(rows_to_add, ignore_index=True)

# Вывод первых нескольких строк для проверки
best_match_df.head(16)



,Пункт требования,Текст требования,Заголовок параграфа,Пункт параграфа,Текст параграфа,Сходство
0,6.1.1,Насосное оборудование (включая вспомогательные...,Общие требования,None,насосное оборудование включая вспомогательные ...,1.000000
1,6.1.3.7,Поскольку в герметичных насосах для охлаждения...,,6.1.3.1,поскольку в герметичных насосах для охлаждения...,0.878083
2,6.1.5,Насосы должны быть предназначены для работы с ...,,6.1.3.1,насосы должны быть предназначены для работы с ...,1.000000
3,6.1.6,Конструкция насоса должна предусматривать возм...,,6.1.3.1,конструкция насоса должна предусматривать возм...,1.000000
4,6.1.7,Насосы должны сохранять свою работоспособность...,,6.1.3.1,б не менее 105 от номинальной частоты вращения...,0.803029
5,6.1.8,Насосы работающие совместно с регулируемым при...,,6.1.3.1,насосы работающие совместно с регулируемым при...,1.000000
6,6.1.25.2,"Если место установки электроопасное, электродв...",Требования по электробезопасности,None,если место установки электроопасное электродви...,0.942793
7,6.1.27,Насос и его привод должны\n удовлетворять крит...,,6.1.25.4,насос и его привод должны удовлетворять критер...,0.977937
8,6.1.31.1,Резьбовые детали должны соответствовать ГОСТ 8...,,6.1.31.1,61311 резьбовые детали должны соответствовать ...,0.851880
9,6.1.31.3,При применении резьб по ГОСТ 8724 (ИСО 261) и ...,,6.1.31.1,при применении резьб по гост 8724 исо 261 и 9 ...,1.000000


#4. Изучение альтернативных способов определения сходства

* **Использование модели BERT**: Для начала, был загружен предобученный токенизатор и модель BERT (bert-base-uncased) из библиотеки Hugging Face. Эта модель является одной из базовых версий BERT, предназначенной для работы с текстами.

* **Работа с текстами**
Списки текстов из параграфов и требований были подготовлены для обработки. Тексты были извлечены из датафрейма best_match_df, который содержал информацию о наилучшем соответствии между параграфами и требованиями.Для каждого списка текстов были получены соответствующие эмбеддинги. С использованием функции косинусного сходства было рассчитано соответствие между параграфами и требованиями.

* Однако полученные результаты оказались чересчур оптимистичными и  малоинформативными.

In [18]:

# Загрузка предобученного токенизатора и модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)

def get_bert_embeddings(texts):
    """Функция для получения BERT эмбеддингов."""
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    with torch.no_grad():
        outputs = model(**encoded_input)
    # Используем [CLS] токен для представления всего текста
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return embeddings

# Подготовка текстов
texts = best_match_df['Текст параграфа'].tolist()
requirements = best_match_df['Текст требования'].tolist()

# Получение эмбеддингов
paragraph_embeddings = get_bert_embeddings(texts)
requirement_embeddings = get_bert_embeddings(requirements)

# Расчет косинусного сходства
cosine_sim = cosine_similarity(paragraph_embeddings, requirement_embeddings)

# Сохранение сходства в DataFrame
for idx in range(len(best_match_df)):
    best_match_df.at[idx, 'Сходство BERT'] = cosine_sim[idx, idx]  # диагональные элементы, если порядок соответствует

# Вывод первых строк DataFrame для проверки
best_match_df.head(20)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

,Пункт требования,Текст требования,Заголовок параграфа,Пункт параграфа,Текст параграфа,Сходство,Сходство BERT
0,6.1.1,Насосное оборудование (включая вспомогательные...,Общие требования,None,насосное оборудование включая вспомогательные ...,1.000000,0.871361
1,6.1.3.7,Поскольку в герметичных насосах для охлаждения...,,6.1.3.1,поскольку в герметичных насосах для охлаждения...,0.878083,0.963336
2,6.1.5,Насосы должны быть предназначены для работы с ...,,6.1.3.1,насосы должны быть предназначены для работы с ...,1.000000,1.000000
3,6.1.6,Конструкция насоса должна предусматривать возм...,,6.1.3.1,конструкция насоса должна предусматривать возм...,1.000000,0.908564
4,6.1.7,Насосы должны сохранять свою работоспособность...,,6.1.3.1,б не менее 105 от номинальной частоты вращения...,0.803029,0.971612
5,6.1.8,Насосы работающие совместно с регулируемым при...,,6.1.3.1,насосы работающие совместно с регулируемым при...,1.000000,0.996048
6,6.1.25.2,"Если место установки электроопасное, электродв...",Требования по электробезопасности,None,если место установки электроопасное электродви...,0.942793,0.959514
7,6.1.27,Насос и его привод должны\n удовлетворять крит...,,6.1.25.4,насос и его привод должны удовлетворять критер...,0.977937,0.837774
8,6.1.31.1,Резьбовые детали должны соответствовать ГОСТ 8...,,6.1.31.1,61311 резьбовые детали должны соответствовать ...,0.851880,0.885295
9,6.1.31.3,При применении резьб по ГОСТ 8724 (ИСО 261) и ...,,6.1.31.1,при применении резьб по гост 8724 исо 261 и 9 ...,1.000000,0.875956


* **Загрузка модели Word2Vec:** Использовалась предобученная модель Word2Vec ('word2vec-google-news-300') из коллекции Google News, загруженная через API gensim.

* **Подготовка модели:** После загрузки убедились, что модель доступна как объект KeyedVectors для использования функции wmdistance, которая позволяет вычислять расстояние по методу Word Mover's Distance (WMD) между двумя текстами.

* **Функция расчёта WMD:** Реализована функция calculate_wmd, которая принимает два текста, преобразует их в нижний регистр, разбивает на слова и вычисляет WMD. Этот метод основан на минимальном расстоянии, необходимом для преобразования всех слов в одном документе в слова другого документа, учитывая их семантическую близость.

* **Применение WMD к данным:** Для каждой пары текстов (требования и соответствующий параграф) в DataFrame best_match_df было вычислено WMD, что позволило оценить их семантическую близость. Результаты сохранены в столбец WMD_Similarity данного DataFrame.

In [ ]:
# Загрузка предобученной модели Word2Vec
model = api.load('word2vec-google-news-300')

# Обеспечение, что модель полностью загружена как KeyedVectors для доступа к функции wmdistance
if not isinstance(model, KeyedVectors):
    model = model.wv

# Функция для вычисления WMD
def calculate_wmd(text1, text2, model):
    text1 = text1.lower().split()
    text2 = text2.lower().split()
    return model.wmdistance(text1, text2)

# Вычисление WMD для каждой пары и добавление в DataFrame
best_match_df['WMD_Similarity'] = best_match_df.apply(lambda row: calculate_wmd(row['Текст требования'], row['Текст параграфа'], model), axis=1)

# Вывод первых нескольких строк для проверки
best_match_df.head(16)


[===-----------------------------------------------] 6.7% 112.0/1662.8MB downloaded

#Интерпретация значений WMD
* Большие значения WMD: Высокое значение WMD указывает на то, что тексты сильно различаются с семантической точки зрения. Слова в этих текстах находятся далеко друг от друга в векторном пространстве, что свидетельствует о большой разнице в содержании или контексте.

* Маленькие значения WMD: Низкое значение WMD означает, что тексты семантически близки. Слова из одного текста находятся близко к словам другого текста в векторном пространстве, что указывает на схожесть или релевантность тем или идей между текстами.

* Нулевое значение WMD: Если WMD равно нулю, это означает, что тексты идентичны с точки зрения используемых слов и их векторных представлений, хотя на практике такое встречается редко, если только тексты не являются полными дубликатами.